In [3]:
#LDA weights 다시 생성하기
#LDA에 모든 데이터를 넣고 그 데이터로 train,test 데이터를 나누기

import pandas as pd
import numpy as np

df = pd.read_pickle('df_noun_2018_re2.pkl')

In [4]:
df.head()

,category,title,mod_content,num_agree,new_status,mod_begin,cont_noun,cont_noun_str
0,행정,아...이런데서 이상한 글이 올라오네요...,"국민청원하는 게시판에서 뭐 섹스,희주야이런 이상한 글 올라오는데 여기는 청원하는데지...",6,0,2018-01-01,"[국민, 청원, 게시판, 섹스, 희, 주야, 글, 청원, 곳, 놈, 처벌, 글]","국민,청원,게시판,섹스,희,주야,글,청원,곳,놈,처벌,글"
1,기타,"현재 사람들 사이에서 유행하는 게임 ""배틀 그라운드"" 에 대한 제재를 가하려 합니다.",배틀 그라운드는 현재 세계적으로나 우리나라 에서나 매우 흥행 하는 게임입니다. 게임...,1,0,2018-01-01,"[배틀, 그라운드, 세계, 나라, 흥행, 게임, 게임, 청소년, 이용, 불가, 게임...","배틀,그라운드,세계,나라,흥행,게임,게임,청소년,이용,불가,게임,표,피시방,청소년,..."
2,정치개혁,양심수 석방 국가보안법 철폐,적폐청산의 시금석 양심수들의 특별사면 배제 문재인 정부를 규탄한다. 지난 문재인 정...,0,0,2018-01-01,"[적폐, 청산, 시금석, 양심수, 특별, 사면, 배제, 문재, 정부, 규탄, 문재,...","적폐,청산,시금석,양심수,특별,사면,배제,문재,정부,규탄,문재,정부,특별,사면,단행..."
3,인권/성평등,민주화유공자들에 대한 국가유공자 인정 요청,"안녕하세요. 살벌한 독재시대를 떠나, 대한민국 민주화를 이룩하신 수많은 열사들이, ...",3,0,2018-01-01,"[안녕, 독재, 시대, 대한민국, 민주, 열사, 민주, 유공자, 인정, 국가, 유공...","안녕,독재,시대,대한민국,민주,열사,민주,유공자,인정,국가,유공자,인정,이야기,청원..."
4,외교/통일/국방,UAE 관련,Uae 괸련한 야당 질의와 관련하여 기본적으로 유시민 작가의 말씀에 동의하여 국익에...,0,0,2018-01-01,"[련, 야당, 질의, 관련, 기본, 유시민, 작가, 말씀, 동의, 국익, 관련, 정...","련,야당,질의,관련,기본,유시민,작가,말씀,동의,국익,관련,정부,문재,정부,참고,야..."


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features = 2000, max_df = 0.80, min_df = 5).fit(df['cont_noun_str'])

In [6]:
cv_array = cv.transform(df['cont_noun_str']).toarray()

In [15]:
print(np.round((now2 - now) / 60, 2))

0.37


In [18]:
#토필모델링 가중치 데이터 생성
from sklearn.decomposition import LatentDirichletAllocation
topic_num = 11

start = time.time()

sk_lda = LatentDirichletAllocation(n_components = topic_num, n_jobs = -1)
sk_lda_f = sk_lda.fit(cv_array)
print('소요시간:', np.round((start - time.time()) / 60, 2))
    #n_jobs =-1 옵션을 썻더니 소요시간이 많이 단축됨

소요시간: -3.24


In [19]:
lda_weights = sk_lda_f.transform(cv_array)

In [20]:
lda_weights.shape
    #26만개의 단어들이 11개의 토픽에 속할 확률을 나탄내는 행렬 생성

(269242, 11)

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(lda_weights, df['category'],
                                                   test_size = 0.3, random_state = 156,
                                                   stratify = df['category'])

In [24]:
#랜덤포레스트로 학습시도
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [26]:
rf_clf = RandomForestClassifier(random_state = 156, n_jobs = -1)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=156)

In [27]:
pred = rf_clf.predict(X_test)
print('LDA weights이용 랜덤포레스트 정확도:', accuracy_score(y_test, pred))
    #정확도가 더욱 하락됨...

LDA weights이용 랜덤포레스트 정확도: 0.34167357904250184


In [28]:
#토픽개수를 8개로 설정해보고 시도해보기
topic_num = 8

start = time.time()

sk_lda = LatentDirichletAllocation(n_components = topic_num, n_jobs = -1)
sk_lda_f = sk_lda.fit(cv_array)
print('소요시간:', np.round((time.time() - start) / 60, 2))

소요시간: 3.53


In [29]:
lda_weights2 = sk_lda_f.transform(cv_array)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(lda_weights2, df['category'],
                                                   test_size = 0.3, random_state = 156,
                                                   stratify = df['category'])

In [31]:
rf_clf = RandomForestClassifier(random_state = 156, n_jobs = -1)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=156)

In [32]:
pred = rf_clf.predict(X_test)
print('LDA weights2이용 랜덤포레스트 정확도:', accuracy_score(y_test, pred))
    #정확도가 더 떨어짐
    #오히려 토픽의 개수를 늘려야 정확도가 늘 수 있나?
    #즉 현재의 토픽 개수는 차원을 줄이는 의미를 주는것이 아닌
    #오히려 정보의 손실을 준다는 의미가 큰것
    #9만개의 단어에 대한 

LDA weights2이용 랜덤포레스트 정확도: 0.28214873782080646


In [33]:
#원래의 17개 카테고리로 토픽개수 설정해보기
topic_num = 17

start = time.time()

sk_lda = LatentDirichletAllocation(n_components = topic_num, n_jobs = -1)
sk_lda_f = sk_lda.fit(cv_array)
print('소요시간:', np.round((time.time() - start) / 60, 2))

소요시간: 3.42


In [34]:
lda_weights3 = sk_lda_f.transform(cv_array)

In [35]:
lda_weights3.shape

(269242, 17)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(lda_weights3, df['category'],
                                                   test_size = 0.3, random_state = 156,
                                                   stratify = df['category'])

In [38]:
rf_clf = RandomForestClassifier(random_state = 156, n_jobs = -1)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=156)

In [39]:
pred = rf_clf.predict(X_test)
print('LDA weights3이용 랜덤포레스트 정확도:', accuracy_score(y_test, pred))
    #5%p 성장
    #아직도 토픽의 개수가 충분치 않은것 같음

LDA weights3이용 랜덤포레스트 정확도: 0.39159124955121144


In [40]:
#한번에 500개 정도로 늘려서 토픽을 설정해보기
topic_num = 500

start = time.time()

sk_lda = LatentDirichletAllocation(n_components = topic_num, n_jobs = -1)
sk_lda_f = sk_lda.fit(cv_array)
print('소요시간:', np.round((time.time() - start) / 60, 2))

소요시간: 11.42


In [41]:
lda_weights4 = sk_lda_f.transform(cv_array)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(lda_weights4, df['category'],
                                                   test_size = 0.3, random_state = 156,
                                                   stratify = df['category'])

In [43]:
rf_clf = RandomForestClassifier(random_state = 156, n_jobs = -1)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=156)

In [44]:
pred = rf_clf.predict(X_test)
print('LDA weights4이용 랜덤포레스트 정확도:', accuracy_score(y_test, pred))
    #토픽의 개수를 1000개로 올리는것이 의미가 있나?
    #현재 문서들이 2000개의 단어로 표현됨
    #-> 거기서 1000개의 주제로 요약하는것은 사실상 단어 2개로 문서를 요약한다는 의미로 해석가능?

LDA weights4이용 랜덤포레스트 정확도: 0.4768672700035903


In [45]:
#단어표현을 5천개로 늘려서 1천개의 토픽으로 요약하기?
cv = CountVectorizer(max_features = 5000, max_df = 0.80, min_df = 5).fit(df['cont_noun_str'])

In [47]:
cv_array2 = cv.transform(df['cont_noun_str']).toarray()

In [48]:
cv_array2.shape

(269242, 5000)

In [49]:
topic_num = 1000

start = time.time()

sk_lda = LatentDirichletAllocation(n_components = topic_num, n_jobs = -1)
sk_lda_f = sk_lda.fit(cv_array2)
print('소요시간:', np.round((time.time() - start) / 60, 2))

소요시간: 40.03


In [51]:
lda_weights5 = sk_lda_f.transform(cv_array2)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(lda_weights5, df['category'],
                                                   test_size = 0.3, random_state = 156,
                                                   stratify = df['category'])

In [53]:
rf_clf = RandomForestClassifier(random_state = 156, n_jobs = -1)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=156)

In [54]:
pred = rf_clf.predict(X_test)
print('LDA weights4이용 랜덤포레스트 정확도:', accuracy_score(y_test, pred))
    #정확도 향상 의미가 없음. 중단

LDA weights4이용 랜덤포레스트 정확도: 0.4822774937169599
